In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd "/content/drive/My Drive/CZ4003"

/content/drive/My Drive/CZ4003


## OCR detection

In [ ]:
import os
import re
import cv2
import glob
from google.colab.patches import cv2_imshow

In [ ]:
if not os.path.exists('./cropped_images'):
  os.mkdir('./cropped_images')

In [ ]:
# Get files
def get_files(folder_name, file_types):
  detected_files = []
  for file_type in file_types:
    print(file_type)
    for file in glob.glob(folder_name + f"/*.{file_type}"):
        detected_files.append(file)
  
  return sorted(detected_files)

def tilted_box_transform(coor):
    x = []
    y = []
    for i in range(len(coor)):
        if i % 2 == 0:
            x.append(coor[i])
        else:
            y.append(coor[i])
    try:
        max_x = max(x)
        min_x = min(x)
        max_y = max(y)
        min_y = min(y)
        new_coor = [min_x, min_y, max_x, min_y, max_x, max_y, min_x, max_y]
    except ValueError:
        return []

    return new_coor

# Extract coordinates of detected bounding boxes from *.txt files
def get_coordinates(file):
    coordinates = []
    rect = []
    file = file.replace(" ", "")
    file = list(file.strip().split("\n"))
    if len(file[0]) != 0: 
        for text in file:
            coor = [int(s) for s in text.split(",")[:8]] #if s[0].isdigit()]
            if text:
                coordinates.append(coor)
    for i in range(len(coordinates)):
        #print(coordinates[i])
        coordinates[i] = tilted_box_transform(coordinates[i])
    
        rect_coor = (coordinates[i][0], coordinates[i][1], coordinates[i][2]-coordinates[i][0], coordinates[i][7]-coordinates[i][1])
        rect.append(rect_coor)

    return rect

In [ ]:
!python ./CRAFT-pytorch-master/test.py --trained_model=./CRAFT-pytorch-master/craft_mlt_25k.pth --test_folder=./sample_images --cuda False

Loading weights from checkpoint (./CRAFT-pytorch-master/craft_mlt_25k.pth)
elapsed time : 14.151941537857056s


In [ ]:
meme_path = "memes/sample02.png"
meme = cv2.imread(meme_path)

In [ ]:
meme_files = get_files('result', ['txt'])
meme_images = get_files('sample_images', ['png','jpg','jpeg'])
print(meme_files)
print(meme_images)
for meme_id in range(len(meme_files)):
  meme = cv2.imread(meme_images[meme_id])
  with open(meme_files[meme_id], 'r') as f: 
    gt = f.read()
  gt_coor = get_coordinates(gt)

  line = 0
  gt_coor_new = []
  for i in range(len(gt_coor)):
    if i != 0:
      if abs(gt_coor[i][1]- gt_coor[i-1][1]) > 10:
        line += 1 
    gt_coor_new.append([line, gt_coor[i]]) 

  bboxes = [] # [line, (bottom_left_x, bottom_left_y, x_width, y_height)]
  for object_id in range(len(gt_coor_new)):
    bboxes.append(gt_coor_new[object_id])
  bboxes = sorted(bboxes , key=lambda k: [k[0], k[1][0], k[1][1]])

  for text_id in range(len(bboxes)):
    p1 = (int(bboxes[text_id][1][0]), int(bboxes[text_id][1][1]))
    p2 = (int(bboxes[text_id][1][0] + bboxes[text_id][1][2]), int(bboxes[text_id][1][1] + bboxes[text_id][1][3]))
    img_crop = meme[p1[1]:p2[1],p1[0]:p2[0]]
    save_path = f'cropped_images/crop_meme_{meme_id}_pt_{text_id}.jpg'
    cv2.imwrite(save_path, img_crop)

txt
png
jpg
jpeg
['result/res_sample02.txt']
['sample_images/sample02.png']


In [ ]:
!python ./deep-text-recognition-benchmark-master/demo.py --Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn --image_folder cropped_images --saved_model ./deep-text-recognition-benchmark-master/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth --sensitive

model input parameters 32 100 20 1 512 256 96 25 TPS ResNet BiLSTM Attn
loading pretrained model from ./deep-text-recognition-benchmark-master/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
cropped_images/crop_meme_0_pt_0.jpg	Sonnet                   	0.9999
cropped_images/crop_meme_0_pt_1.jpg	for                      	1.0000
cropped_images/crop_meme_0_pt_2.jpg	Lena                     	0.9996
cropped_images/crop_meme_0_pt_3.jpg	0                        	0.4729
cropped_images/crop_meme_0_pt_4.jpg	dear                     	0.9893
cropped_images/crop_meme_0_pt_5.jpg	Lenu,                    	0.4478
cropped_images/crop_meme_0_pt_6.jpg	your                     	0.9991
cropped_images/crop_meme_0_pt_7.jpg	beauly                   	0.8301
cropped_images/crop_meme_0_pt_8.

In [ ]:
word_cf_dict = {} # id: [word, confidence_score]
sentence_dict = {}
with open("./log_demo_result.txt", 'r') as f: 
  reg = f.read()
reg = list(reg.strip().split("\n"))
words = []
sentence = ''
for text in reg[3:]:
  text_id_old = text_id
  text_list = text.split("\t")
  file_name = text_list[0]
  img_no = re.search('meme_(.+?)_', file_name)
  if img_no:
      text_id = int(img_no.group(1))
  if text_id != text_id_old:
    words = []
    sentence = ''
  # print(type(text_list[1]))
  word = text_list[1]
  cf = float(text_list[2])
  if text:
    word = word.strip()
    words.append([word, cf])
    sentence += word + ' '
  word_cf_dict[text_id] = words 
  sentence_dict[text_id] = sentence

In [ ]:
sentence_dict

{0: "Sonnet for Lena 0 dear Lenu, your beauly is so vast It is hard somet lines to describe it fast, I thought the entire world I would impress the only your portrait I could compross. Alas! First when I tried to use Vq I found that your checks belong to only you. Your silky hair contains a thousand lines Hard to match with sums of discrete cosines. And for your lips, sensual and tactual Thirteen Crays found not the proper fractal. And while these set backs are all quite sover a might have fixed them with hacks here or there But when filters took sparkle from your eyes I said, Damn all this. I'll just digitize. Thomas Colth nost "}